In [ ]:
# winequality-red.csv, winequality-white.csv 파일의 데이터를 읽어서 테이블에 저장
# 1. 각 파일의 데이터를 읽어서 각각 type 컬럼을 만들고 'red', 'white' 저장
# 2. 두 파일의 데이터를 병합
# 3. 데이터베이스는 -2.ipynb에서 만든 data_repo 사용 (새로 만들기 X)
# 4. 테이블 만들기 (red_wine -> wine)
# 5. 데이터 읽어서 wine 테이블에 저장
# 6. 7등급 이상의 데이터를 조회해서 파일로 저장 (high-quality-wine.csv)

In [2]:
# 데이터 분석 패키지 준비
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
# 1. 각 파일의 데이터를 읽어서 각각 type 컬럼을 만들고 'red', 'white' 저장
df_red = pd.read_csv('data-files/winequality-red.csv', sep=";")
df_white = pd.read_csv('data-files/winequality-white.csv', sep=";")

print( df_red.columns, df_white.columns ) # 컬럼 동일성 확인

df_red['type'] = 'red'
df_white['type'] = 'white'


Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object') Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')


In [8]:
# 2. 두 파일의 데이터를 병합
df_combined = pd.concat([df_red, df_white])
print( df_red.shape, df_white.shape, df_combined.shape)

(1599, 13) (4898, 13) (6497, 13)


In [9]:
# 데이터베이스 연동 패키지(모듈) 준비
import pymysql

In [11]:
# 2. 테이블 만들기

try:
    # 데이터베이스 연결
    conn = pymysql.connect(
        host="127.0.0.1",
        user="humanda5",
        password="humanda5",
        database="data_repo"
    )

    # 명령 실행기 준비
    cursor = conn.cursor()

    # 명령 실행
    sql = "drop table if exists wine"
    cursor.execute(sql)
    sql = """create table if not exists wine
          (
            idx int primary key auto_increment,
            fixed_acidity float not null,
            volatile_acidity float not null,
            citric_acid float not null,
            residual_sugar float not null,
            chlorides float not null,
            free_sulfur_dioxide float not null,
            total_sulfur_dioxide float not null,
            density float not null,
            pH float not null,
            sulphates float not null,
            alcohol float not null,
            quality int not null,
            type varchar(10) not null
          )"""
    cursor.execute(sql)    
except Exception as e:
    print("오류 발생 : ", e)
finally:
    # 연결 종료
    if cursor:
        cursor.close()
    if conn:
        conn.close()

In [12]:
# 3. 파일 데이터 읽어서 위에서 만든 테이블에 저장 

try:
    # 데이터베이스 연결
    conn = pymysql.connect(
        host="127.0.0.1",
        user="humanda5",
        password="humanda5",
        database="data_repo"
    )

    # 명령 실행기 준비
    cursor = conn.cursor()

    # 명령 실행
    sql = "delete from wine"
    cursor.execute(sql)
    sql = """insert into wine 
             (fixed_acidity, volatile_acidity, citric_acid, residual_sugar, chlorides, free_sulfur_dioxide, total_sulfur_dioxide, density, pH, sulphates, alcohol, quality, type)
             values 
             (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    data = df_combined.values.tolist()
    cursor.executemany(sql, data)

    conn.commit() # 마지막 commit or rollback 이후에 실행된 모든 변경사항 확정
except Exception as e:
    print("오류 발생 : ", e)
    if conn:
        conn.rollback() # 마지막 commit or rollback 이후에 실행된 모든 변경사항 취소
finally:
    # 연결 종료
    if cursor:
        cursor.close()
    if conn:
        conn.close()

In [18]:
# 4. 데이터베이스의 데이터 중 quality가 7 이상인 데이터를 무작위 조회해서 파일로 저장

try:
    # 데이터베이스 연결
    conn = pymysql.connect(
        host="127.0.0.1",
        user="humanda5",
        password="humanda5",
        database="data_repo"
    )

    # 명령 실행기 준비
    cursor = conn.cursor()

    # 명령 실행
    sql = """select * from wine where quality >= %s""" # lower : 소문자로 변경
    cursor.execute(sql, [7])

    wine_data = cursor.fetchall() # 조회 결과를 tuple 형식으로 반환

    # 파일로 저장
    df_wine = pd.DataFrame(wine_data, 
                           columns=['idx'] + list(df_combined.columns))
    # df_iris.drop(['idx'], axis=1) # 'idx' 컬럼 제거, 원본 유지, 변경된 복사본 반환
    df_wine.drop(['idx'], axis=1, inplace=True) # 'idx' 컬럼 제거, 원본 변경
    df_wine.to_csv('data-files/high-quality-wine.csv', index=False) # index 저장하지 않도록 설정
    
except Exception as e:
    print("오류 발생 : ", e)

finally:
    # 연결 종료
    if cursor:
        cursor.close()
    if conn:
        conn.close()

In [20]:
print(['idx'] + df_combined.columns) # df_combined.columns의 각 요소에 idx를 결합
print(['idx'] + list(df_combined.columns)) # 두 리스트 병합

Index(['idxfixed acidity', 'idxvolatile acidity', 'idxcitric acid',
       'idxresidual sugar', 'idxchlorides', 'idxfree sulfur dioxide',
       'idxtotal sulfur dioxide', 'idxdensity', 'idxpH', 'idxsulphates',
       'idxalcohol', 'idxquality', 'idxtype'],
      dtype='object')
['idx', 'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality', 'type']


In [17]:
['idx'] + list(df_combined.columns)

['idx',
 'fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality',
 'type']